In [1]:
import openai
import os
import json
import tiktoken
from pydantic import BaseModel
from typing import Optional
from dotenv import load_dotenv

# .env 파일에서 API 키 로드
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# 모델 파라미터 로드 함수
def load_model_parameters(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        parameters = json.load(file)
    return parameters

# 파일 경로에 따라 교과서 내용을 로드하는 함수
def load_textbook_content_from_input(
    main_chapter, sub_chapter, small_chapter, base_dir="./data/textbook/"
):
    file_name = f"{main_chapter}_{sub_chapter}_{small_chapter}.txt"
    file_path = os.path.join(base_dir, file_name)

    # 파일이 존재하는지 확인하고 내용을 읽어오기
    if os.path.isfile(file_path):
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()
        return content
    else:
        return f"파일을 찾을 수 없습니다: {file_path}"

In [7]:
# 요청에 대한 데이터 모델 정의
class RequestData(BaseModel):
    grade: int  # 중1, 중2, 중3, 전체 등의 학년 선택
    subject_area: str  # 영역 선택 (예: 도덕)
    main_chapter: int  # 대단원
    sub_chapter: int  # 중단원
    small_chapter: int  # 소단원
    requirement: Optional[str]  # 요청사항 (선택 사항)

# GPT API를 사용하여 학습 활동을 추천하는 함수
def recommend_learning_activities_in_korean(textbook_content, data: RequestData, model_parameters):
    prompt = f"""
다음은 응답의 기본이 될 교과서의 내용입니다:
{textbook_content}

다음은 사용자의 요청 정보입니다:
- 학년: 중학교 {data.grade}학년
- 과목: {data.subject_area}
- 요청사항: {data.requirement if data.requirement else '없음'}

**요청 사항이 있는 경우, 반드시 요청 사항을 우선적으로 반영하세요**

교사의 수업 중 다양한 에듀테크 도구들을 수업에 활용할 수 있도록 장려하려고 합니다.
교과서의 학습 활동 내용과 사용자 요청을 바탕으로, 중학생들의 수업 시간에 적용할 수 있는 에듀테크 기반의 학습 활동을 제안해 주세요.

**응답은 다음의 마크다운 형식을 따라 주세요:**

## 교과
[교과명]

## 대상
[대상 학년]

## 단원
[단원명]

## 준비물
[준비물이 필요한 경우 제시해주세요.]

## 수업 방식
[수업 방식]

## 에듀테크 제품
[제품명 및 추천 이유/활용 방안]

## 수업 목표
- 목표

## 단계별 학습활동
아래 표에 각 단계별로 구체적인 학습활동, 활동의 목적, 진행 방법, 학생들의 역할, 그리고 수업형태를 상세히 작성해 주세요. 추천한 에듀테크 제품을 활동 중 적절하게 사용할 수 있도록 창의적인 답변을 주세요.

| **단계** | **학습활동** | **수업형태** |
|---|---|---|
| 도입 | [예: 학생들의 관심을 끌기 위한 동기 부여 활동] | [수업형태] |
| 전개1 | [예: 에듀테크 도구를 활용한 핵심 개념 학습] | [수업형태] |
| 전개2 | [예: 그룹 활동을 통한 문제 해결] | [수업형태] |
| 마무리 | [예: 학습 내용 정리 및 피드백] | [수업형태] |


## 활용효과
- 효과
"""
    try:
        # ChatCompletion 생성 요청
        response = openai.ChatCompletion.create(
            model=model_parameters["model_name"],  # 모델 이름
            messages=[
                {"role": "system", "content": "당신은 중학생 대상 교육 활동 지도가이자 에듀테크 컨텐츠 추천 전문가입니다."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=model_parameters["max_tokens"],  # 최대 토큰 수
            temperature=model_parameters["temperature"],  # 응답의 무작위성 제어
            top_p=model_parameters["top_p"],  # 확률 분포 제어
            n=model_parameters["n"],  # 생성할 응답의 수
            presence_penalty=model_parameters["presence_penalty"],  # 새로운 주제 생성 유도
            frequency_penalty=model_parameters["frequency_penalty"],  # 반복 내용 생성 방지
            request_timeout=model_parameters["request_timeout"],  # 요청 시간 제한 (없으면 None)
            # logit_bias=model_parameters["logit_bias"]  # 필요 시 사용
        )
        return response.choices[0].message['content'].strip()
    except Exception as e:
        return f"Error: {str(e)}"


In [8]:
test_data = RequestData(
        grade=2,
        subject_area='도덕',
        main_chapter=3,
        sub_chapter=1,
        small_chapter=2,
        requirement='사회적 약자의 불편함에 대해 공감하고 해결책을 논의할 수 있도록 모둠 활동 컨텐츠를 추천해줘. '
    )

model_parameters = load_model_parameters('model.json')
textbook_content = load_textbook_content_from_input(test_data.main_chapter,test_data.sub_chapter, test_data.small_chapter)


activities = recommend_learning_activities_in_korean(textbook_content, test_data, model_parameters)
print(activities)

## 교과
도덕

## 대상
중학교 2학년

## 단원
02 사회적 약자를 어떻게 대할 것인가?

## 준비물
없음

## 수업 방식
그룹 활동

## 에듀테크 제품
- Padlet: 학생들이 의견을 공유하고 토론할 수 있는 온라인 게시판 플랫폼입니다.
- Google 도큐먼트: 학생들이 그룹 활동의 결과물을 공유하고 편집할 수 있는 협업 도구입니다.

## 수업 목표
- 사회적 약자의 불편함과 어려움에 대해 공감하고 이해합니다.
- 사회적 약자를 위한 해결책을 모둠 활동을 통해 논의하고 제안합니다.

## 단계별 학습활동

| **단계** | **학습활동** | **수업형태** |
|---|---|---|
| 도입 | - 학생들에게 사회적 약자의 어려움과 불편함에 대해 이야기하고 공감을 유도합니다.<br>- "사회적 약자의 입장에서 생활해보기"라는 주제로 학생들에게 다양한 장애를 시뮬레이션하는 활동을 시행합니다. 예를 들어, 학생들에게 손을 묶거나 시력을 가리는 등의 제약을 가하고 일상 활동을 수행하도록 합니다. | 전체 수업 시간 |
| 전개1 | - 학생들을 그룹으로 나누고 각 그룹에게 사회적 약자의 어려움을 해결하기 위한 제안을 하도록 합니다.<br>- 각 그룹은 Padlet을 사용하여 자신들의 제안을 기록하고 공유합니다. | 그룹 활동 |
| 전개2 | - 각 그룹은 다른 그룹의 제안을 검토하고 피드백을 제공합니다.<br>- Google 도큐먼트를 사용하여 그룹별로 제안을 수정하고 보완합니다. | 그룹 활동 |
| 마무리 | - 각 그룹은 자신들의 최종 제안을 발표하고 토론합니다.<br>- 학생들은 다른 그룹의 제안에 대해 의견을 나누고 피드백을 제공합니다.<br>- 수업의 마무리로 학생들은 사회적 약자를 위한 해결책에 대한 자신의 생각을 Google 도큐먼트에 정리합니다. | 전체 수업 시간 |

## 활용효과
- 학생들은 사회적 약자의 어려움과 불편함에 대해 공감하고 이해할 수 있습니다.
- 그룹 활동을 통해 학생들은 사회적 약자를 위한 해결책을 